In [6]:
! pip install -qU openai datasets


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [4]:
import getpass
from openai import OpenAI


In [8]:
import os
os.environ['OPENAI_API_KEY'] = getpass.getpass('Lấy API key nhập vào đây')

In [10]:
from datasets import load_dataset
dataset = load_dataset("gaodrew/midjourney-prompts-highquality",split="train")

selected_dataset = dataset.select(range(1000))

README.md:   0%|          | 0.00/881 [00:00<?, ?B/s]

(…)journey_prompts_90thpercentilelength.csv:   0%|          | 0.00/153M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/94438 [00:00<?, ? examples/s]

In [13]:
import re
endhance_promts_raw = selected_dataset['content']
prompt_pairs = []
def extract_prompts(content):
    match = re.findall(r'(?<=Prompt: ).*?(?=\n)', content)
    if match:
        extracted_prompts = match.group(1)
        return extracted_prompts
    
for text in endhance_promts_raw:
    prompt_pairs.append( {
        "extract_prompts": extract_prompts(text)
    })


In [16]:
from openai import OpenAI
from tqdm.auto import tqdm
import json
import time
from tenacity import retry, stop_after_attempt, wait_exponential

client = OpenAI()

@retry(stop=stop_after_attempt(5), wait=wait_exponential(multiplier=1, min=2, max=10))
def get_simple_prompt(enhanced_prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a customer service representative helping a customer with a problem."},
            {"role": "user", "content": f"ENHANCE PROMPT: {enhanced_prompt}; SIMPLE BASIC PROMPT:"}
        ]
    )
    return response.choices[0].message.content

prompt_pairs = [
    {"extracted_prompts": "How can I reset my password?"},
    {"extracted_prompts": "What are your refund policies?"}
]

for prompt in tqdm(prompt_pairs):
    try:
        prompt['simple_prompt'] = get_simple_prompt(prompt['extracted_prompts'])
    except Exception as e:
        prompt['simple_prompt'] = f"Error: {str(e)}"

with open('simple_prompts.json', 'w') as f:
    json.dump(prompt_pairs, f, indent=4)

# Uncomment the following lines if running in Google Colab
# from google.colab import files
# files.download('simple_prompts.json')

  0%|          | 0/2 [00:00<?, ?it/s]

In [22]:
import json
from typing import List, Dict

FILE_PATH = "training_data.jsonl"

def save_training_data(data: List[Dict], output_file: str = FILE_PATH):
    """
    Save training data to a JSONL file (JSON Lines format)

    Args:
    data: List[Dict]: List of dictionaries containing the training data
    output_file: str: Output file path
    """
    with open(output_file, 'a') as f:
        for item in data:
            f.write(json.dumps(item) + "\n")

# Load existing data from JSONL file if it exists
def load_training_data(file_path: str = FILE_PATH) -> List[Dict]:
    try:
        with open(file_path, 'r') as f:
            return [json.loads(line) for line in f]
    except (FileNotFoundError, json.JSONDecodeError):
        return []

# Example of adding new data
def add_training_data(new_data: List[Dict]):
    existing_data = load_training_data()
    existing_data.extend(new_data)
    save_training_data(new_data)  # Chỉ ghi dữ liệu mới, không ghi lại toàn bộ file

# Example usage
prompt_pairs = [
    {"question": "How to reset my password?", "answer": "Go to settings and click reset password."},
    {"question": "What is the refund policy?", "answer": "You can request a refund within 30 days."}
]

add_training_data(prompt_pairs)

In [24]:
example_data = prompt_pairs[0]

message =[{"role": "system", "content": "You are a customer service representative helping a customer with a problem."},
          {"role": "user", "content": f"ENHANCE PROMPT: {example_data['question']}; SIMPLE BASIC PROMPT: {example_data['answer']}"}]

message

[{'role': 'system',
  'content': 'You are a customer service representative helping a customer with a problem.'},
 {'role': 'user',
  'content': 'ENHANCE PROMPT: How to reset my password?; SIMPLE BASIC PROMPT: Go to settings and click reset password.'}]